In [2]:
from diffusers import StableDiffusionXLPipeline, EulerAncestralDiscreteScheduler
import torch

model_id = "aipicasso/emi"
freeu_params = dict(b1=1.1, b2=1.2, s1=0.6, s2=0.4)
# freeu_params = dict(b1=1.5, b2=1.6, s1=0.9, s2=0.2) # SD1.5 詳細が消える
# freeu_params = dict(b1=1.3, b2=1.4, s1=0.9, s2=0.2) # SD1.4 詳細がやや消える・色がまだおかしいけど、ましになった

# model_id = "aipicasso/emi-2"
# freeu_params = dict(s1=1.2, s2=0.7, b1=1.1, b2=1.3)

dtype = torch.float32
# dtype = torch.float16
scheduler = EulerAncestralDiscreteScheduler.from_pretrained(model_id, torch_dtype=dtype, subfolder="scheduler")
pipe = StableDiffusionXLPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=dtype)
pipe = pipe.to("cuda")
pipe.enable_freeu(**freeu_params)


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
# 職業
jobs = sum([
    ["knight"]   * 20,
    ["noble"]    * 20,
    ["swordman"] * 10,
    ["archer"]   * 5,
    ["warrior"]  * 20,
    ["thief"]    * 1,
    ["priest"]   * 1,
    ["mage"]     * 1,
    ["merchant"] * 1,
    ["pirate"]   * 1,
    ["NONE"]     * 10,
], [])


# 性別
jenders = sum([
    ["1boy"]                * 10,
    ["1boy, (shota)"]         * 10,
    ["1boy, teenage"]       * 10,
    ["1boy, adult"]         * 10,
    ["1girl"]               * 10,
    ["1girl, (loli)"]         * 10,
    ["1girl, teenage"]      * 10,
    ["1girl, adult"]        * 10,
    ["solo, mature male"]   * 5,
    ["solo, mature female"] * 5,
    ["solo, old man"]       * 3,
    ["solo, old woman"]     * 3,
    ["solo, facial hair"]   * 3,
], [])


hair_colors = sum([
    ["black hair"]  * 5,
    ["blond hair"]  * 5,
    ["brown hair"]  * 5,
    ["red hair"]    * 4,
    ["green hair"]  * 4,
    ["blue hair"]   * 4,
    ["silver hair"] * 1,
    ["purple hair"] * 1,
    ["pink hair"]   * 1,
    ["NONE"]        * 1,
], [])

# 髪型
hair_types = sum([
    ["long hair"]  * 1,
    ["short hair"] * 1,
    ["NONE"]       * 8,
], [])


# 装飾
ornaments = sum([
    ["hair ornament"] * 1,
    ["ribbon"]        * 1,
    # ["glasses"]       * 1,
    ["scarf"]         * 1,
    # ["gloves"]        * 1,
    ["sword"]         * 1,
    ["bow"]           * 1,
    # ["eyepatch"]      * 1,
    ["scar"]          * 1,
    ["NONE"]          * 10,
], [])

# 性格
personalities = sum([ 
    ["calm"]        * 2,
    ["brave"]       * 5,
    ["intelligent"] * 1,
    ["smile"]       * 1,
    ["charming"]    * 1,
    ["strong"]      * 1,
    ["serious"]     * 1,
    ["generous"]    * 1,
    ["bright"]      * 1,
    ["vivid"]       * 1,
    ["NONE"]        * 0,
], [])



In [ ]:
# from pathlib import Path
# import numpy as np
# path = Path(r"outputs\aaaaaaaaaaaaaaaaaaa\良い - 重複削除")
# valid_params3 = np.unique([tuple(map(lambda s: float(s[3:]), p.name[:35].split(" ")))[1:] for p in path.glob("*")], axis=0)
# cand_params = [
#     (b1, *p3)
#     for b1 in np.linspace(1, 1.2, 3)
#     for p3 in valid_params3
# ]
# cand_params


import random
def pick_one(arr):
    one = random.choice(arr)
    return one

def generate_prompt():
    jender = pick_one(jenders)
    hair_color = pick_one(hair_colors)
    randoms = [
        pick_one(jobs),
        jender,
        hair_color,
        pick_one(hair_types),
        pick_one(ornaments),
        pick_one(personalities),
    ]
    randoms = [r for r in randoms if r != "NONE"]
    random_prompt = ", ".join(randoms)
    # prompt = f"anime artwork, anime style, (portrait), {random_prompt}, dark skin, medieval, fantasy, ((transparent))"
    prompt = f"anime artwork, anime style, (portrait), {random_prompt}, medieval, fantasy, ((transparent))"
    return prompt, jender, hair_color

from pathlib import Path
out_dir = Path("D:\\outputs")
out_dir.mkdir(exist_ok=True)
image_id = 0

from tqdm.auto import tqdm


for i in tqdm(range(100000)):
    prompt, jender, hair = generate_prompt()
    is_female = ("female" in jender) or ("girl" in jender) or ("woman" in jender)

    # para = pick_one(cand_params)
    # para = dict(zip(["b1", "b2", "s1", "s2"], para))
    # pipe.enable_freeu(**para)
    # para_str = " ".join([f"{k}-{para[k]:.3f}" for k in para])
    para_str = ""

    file_prefix = ("f" if is_female else "m") + "-" + hair
    filename = f"{file_prefix}__{prompt}__{para_str}__{image_id:05d}.png"

    out_path = out_dir / "current" / file_prefix / filename
    out_path.parent.mkdir(exist_ok=True, parents=True)
    image_id += 1

    nega = "(embedding:unaestheticXLv31:0.5), photo, deformed, realism, disfigured, low contrast, bad hand"
    image = pipe(
        prompt=prompt,
        negative_prompt=nega,
        num_inference_steps=20,
    ).images[0]
    image.save(out_path)


In [ ]:
# 以下の範囲をグリッドサーチする。
# b1: 1 ≤ b1 ≤ 1.2
# b2: 1.2 ≤ b2 ≤ 1.6
# s1: 0.2 ≤ s1 ≤ 1
# s2: 0.2 ≤ s2 ≤ 1

import numpy as np
b1 = np.linspace(1, 1.2, 3)
b2 = np.linspace(1.0, 1.6, 5)
s1 = np.linspace(0.2, 1, 4)
s2 = np.linspace(0.2, 1, 4)

params_list = [
    dict(b1=b1_, b2=b2_, s1=s1_, s2=s2_)
    for b1_ in b1
    for b2_ in b2
    for s1_ in s1
    for s2_ in s2
]

g = torch.Generator(device="cuda").manual_seed(42)
prompt, jender, hair = generate_prompt()
for para in tqdm(params_list):
    print(para)
    pipe.enable_freeu(**para)

    g = g.manual_seed(42)
    prompt = "anime artwork, anime style, (portrait), knight, 1girl, adult, blond hair, scarf, brave, medieval, fantasy, ((transparent))"
    for i in range(4):
        #     prompt, jender, hair = generate_prompt()
        #     is_female = ("female" in jender) or ("girl" in jender) or ("woman" in jender)
        filename = f"{prompt}__{image_id:05d}.png"
        out_path = out_dir / "current" / (" ".join([f"{k}-{para[k]:.3f}" for k in para]) + "__" + filename)
        # out_path = out_dir / "current" / (("f" if is_female else "m") + "-" + hair) / filename
        out_path.parent.mkdir(exist_ok=True, parents=True)
        image_id += 1

        nega = "(embedding:unaestheticXLv31:0.5), photo, deformed, realism, disfigured, low contrast, bad hand"
        image = pipe(
            prompt=prompt,
            negative_prompt=nega,
            num_inference_steps=20,
            generator=g
        ).images[0]
        image.save(out_path)

In [ ]:

# for xx in range(20):
#     filename = f"{prompt}__{image_id:05d}.png"
#     out_path = out_dir / filename
#     image_id += 1

#     nega = "(embedding:unaestheticXLv31:0.5), photo, deformed, realism, disfigured, low contrast, bad hand"
#     image = pipe(
#         prompt=prompt,
#         negative_prompt=nega,
#         num_inference_steps=20,
#         seed=42,
#     ).images[0]
#     image.save(out_path)
#     image